In [1]:
import pandas as pd
import numpy as np
import scipy as sp

from itertools import chain

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
#read in v3 data which is a dictionary of aa and what do they stand for
v3 =pd.read_csv('~/Documents/persona_project_cardiology/original data and supplements/v3_segments.csv')

In [ ]:
#read in number of visit by month
num_month =pd.read_csv('~/Documents/persona_project/result_num_visit.csv')
num_month.columns=['DEVICE_ID_HASHED','jan_visit','feb_visit','mar_visit','apr_visit','may_visit', 'june_visit','july_visit','august_visit']
num_month=num_month.fillna(0)

In [6]:
#find segment ID basic information
aa=v3.SEGMENT_ID.unique()
round(v3.SEGMENT_ID.describe(),2)

count        4262.00
mean      1798324.32
std       3995122.71
min         18387.00
25%         94615.25
50%         95691.50
75%         96756.75
max      19698717.00
Name: SEGMENT_ID, dtype: float64

In [7]:
#read in persona data
persona =pd.read_csv('~/Documents/persona_project_cardiology/original data and supplements/persona_exploration.csv')

In [8]:
persona.shape

(58591, 7)

In [9]:
len(persona.DEVICE_ID_HASHED.drop_duplicates())

58587

In [10]:
#only AA in persona data
aa_persona=persona.AA

In [11]:
#split AA by comma
aa_persona_a=[]
for i in range(len(aa_persona)):
    aa_persona_a.append(aa_persona[i].split(","))

In [12]:
#unlist AA
aa_persona_b=list(chain(*aa_persona_a))

#change AA from list to array
aa_persona_c=np.asarray(aa_persona_b)

#remove missing (nothing is gone, but just make sure)
#aa_persona_all_2 = aa_persona_all_2[~pd.isnull(aa_persona_all_2)]

#put AA into a table 
aa_df=pd.DataFrame(aa_persona_c)
aa_df=aa_df.rename({0: 'aa'}, axis='columns')

In [13]:
#find AA and its associated frequency
unique_elements, counts_elements = np.unique(aa_df, return_counts=True)

aa_freq=pd.DataFrame(unique_elements, counts_elements)
aa_freq.reset_index(level=0, inplace=True)
aa_freq.columns=['freq','aa']

#sort by frequency
aa_freq=aa_freq.sort_values(by=['freq'], ascending=False)

In [14]:
aa_freq

,freq,aa
3906,33312,96844
67,29623,14700004
71,29393,14700010
59,29349,14699992
63,29288,14699998
75,29103,14700016
55,29024,14699986
87,27825,14700032
228,27374,14700218
143,26224,14700115


In [15]:
#change aa data type to int 
aa_freq.aa=aa_freq.aa.astype('int')

In [ ]:
plt.hist(aa_freq.freq)
plt.show()
#most of the aa only appeared less than 5000 times

In [ ]:
#describe the frequency of AA
aa_freq.freq.describe(percentiles=[.25, .5, .75, 0.9, 0.99])

In [18]:
#merge aa with segment info
aa_freq2=aa_freq.merge(v3, left_on='aa', right_on='SEGMENT_ID', how='left')

In [19]:
#take a subset of data that has AA larger than or equal to certain frequency (here top 10% frequency)
#aa_sub90=aa_freq2[aa_freq2['freq']>=7347]
#aa_sub90.iloc[101:150,:]
#find the unique higher category information
#aa_sub90.CATEGORY3.unique()

In [20]:
aa_sub99=aa_freq2[aa_freq2['freq']>=4687]
aa_sub99['cat']=aa_sub99.CATEGORY2.astype('str')+' '+aa_sub99.CATEGORY3.astype('str')+' '+aa_sub99.SEGMENT_NAME.astype('str')

/Users/aiyizhang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
#dataframe with segment IDs and their associated names/cats
aa_sub99b=aa_sub99.loc[:,['SEGMENT_ID', 'freq', 'cat']]

In [24]:
aa_sub99b.to_csv('~/Documents/persona_project_cardiology/clustering/names.csv')

In [ ]:
#find selected aa in persona table (can't merge, because aa in persona is in long string)
#(group a has these top common words and group b does not have these top common words)

In [ ]:
#create empty new dataframe indicating if device has each of the most common features or not
id=aa_sub99b.SEGMENT_ID.astype('str')
id_str='ID_'+aa_sub99b.SEGMENT_ID.astype('str')
ind=pd.DataFrame(0, index=persona.DEVICE_ID_HASHED, columns=id_str)

In [ ]:
#fill in the ind table. if id in 
for j in range(persona.shape[0]):
    for i in range(len(id)):
        if id[i] in persona.AA[j]:
            ind.iat[j,i]=1
        else:
            ind.iat[j,i]=0

In [ ]:
#merge original persona data with new AA id indicators 
persona1=persona.merge(ind, left_on='DEVICE_ID_HASHED', right_on='DEVICE_ID_HASHED')

In [ ]:
persona1.to_csv('persona1_processed.csv')

In [ ]:
#persona1.iloc[0:100,:]

In [ ]:
#only focus on the features from AA, and forget about the number of visit per month for now. 

#join with number of visits per month per device 
#persona2=persona1.merge(num_month,left_on='DEVICE_ID_HASHED', right_on='DEVICE_ID_HASHED')

#because the data are not the whole samle, the device ID do not match in aa table and num_visit table. 
#take inner join to make the dataset smaller, and easier to work with. 
#will change after able to work with the whole dataset 

#persona2.shape

In [ ]:
#persona2b=persona2.iloc[:,2:]

In [ ]:
#persona2b.index=persona2.DEVICE_ID_HASHED

In [ ]:
#persona2c=persona2b.drop(columns=['SEGMENT_ID'])
#persona2c=persona2c.drop(columns=['INCOME_RANGE'])
#persona2c=persona2c.drop(columns=['AGE_RANGE'])